In [43]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
from tqdm import tqdm
import time
from datetime import datetime
import copy

In [44]:
# MongoDB 클라이언트 및 컬렉션 설정
client = MongoClient(host="1.234.51.110", port=38019, username='clawling', password='goodtime**95')
db = client['BOF2024_crawling']
collection = db['tiktok_crawling']
data_list = list(collection.find({"검색키워드": "釜山亚洲艺术节"}, {"_id": 1, "URL": 1}))

In [45]:
data_list1 = copy.deepcopy(data_list)

In [46]:
driver = webdriver.Chrome()

In [47]:
def process_link(url):
    try:
        driver.get(url)
        time.sleep(5)   # 페이지가 로드될 때까지 대기

        # 영상 길이 추출
        try:
            time_element = driver.find_element(By.CSS_SELECTOR, 'div.css-1cuqcrm-DivSeekBarTimeContainer')
            time_text = time_element.text
            if '/' in time_text:
                video_length = time_text.split('/')[1].strip()
            else:
                video_length = ""
        except Exception as e:
            print(f'영상 길이 추출 오류: {e}')
            video_length = ""

        # 좋아요 개수 추출
        try:
            like_count_element = driver.find_element(By.CSS_SELECTOR, 'strong[data-e2e="like-count"]')
            like_count_text = like_count_element.text
            if 'K' in like_count_text:
                like_count = int(float(like_count_text.replace('K', '')) * 1000)
            elif 'M' in like_count_text:
                like_count = int(float(like_count_text.replace('M', '')) * 1000000)
            else:
                like_count = int(like_count_text)
        except Exception as e:
            print(f'좋아요 개수 추출 오류: {e}')
            like_count = ""

        # MongoDB에 적재
        collection.update_one({"URL": url}, {"$set": {
            "영상 길이": video_length,
            "좋아요": like_count,
        }})
    
    except Exception as e:
        print(f'오류 발생: {e}')

In [ ]:
for data in tqdm(data_list1, desc="Processing Links", unit="link"):
    url = data['URL']
    process_link(url)

client.close()
driver.quit()